In [ ]:
I have a Table for Customers by name : Customer_SCD in SQL I have 3 Columns present in it : Customer_Name, 
Customer_ID Customer_TimeStamp

There are duplicate entries in this table with different Timestamp.

For example

ABC, 1, 2012-12-05 11:58:20.370

ABC, 1, 2012-12-03 12:11:09.840

I want to eliminate this from the database and keep the first time/date available.



In [ ]:
Remember we are deleting records with duplicate timestamps keeping only the min(timestamp), i.e., the first timestamp

DELETE Customer_SCD
OUTPUT deleted.*#means whatever records affected by the query statement
from Customer_SCD b
join

(
select a.Customer_Name as cust_name, a.Customer_ID as cust_id, min(a.Customer_TimeStamp) as cust_timestamp
from Customer_SCD a
group by a.Customer_Name, a.Customer_ID
) fog

on b.cust_id=fog.cust_id
and b.cust_id=fog.cust_id
and b.Customer_TimeStamp<>fog.cust_timestamp;


In [ ]:
Alternative solution row_number


DELETE  Customer_SCD
OUTPUT  deleted.*
FROM    Customer_SCD b
JOIN    (
    SELECT  Customer_ID,
            Customer_Name,
            Customer_TimeStamp,
            ROW_NUMBER() OVER (PARTITION BY Customer_ID, Customer_Name ORDER BY Customer_TimeStamp) num
    FROM    Customer_SCD
) c ON 
    c.Customer_ID = b.Customer_ID
AND c.Customer_Name = b.Customer_Name
AND c.Customer_TimeStamp = b.Customer_TimeStamp
AND c.num <> 1 #which means except the row number 1 delete the rest from the table


In [ ]:
This is in SQL Server

The TIMESTAMP datatype in SQL Server (and SQL Azure) is NOT what the ISO/ANSI Standard defines 
(this is a leftover of the original Sybase heritage of SQL Server).

It's really just a binary counter for optimistic concurrency checking - it has NOTHING to do with a date and/or time 
at all!

To store and handle dates and times, use the datatype DATE (for just dates - no time) or DATETIME2(n) for date&time 
instead



In [ ]:


The DATETIME type is used when you need values that contain both date and time information. MySQL retrieves and 
displays DATETIME values in 'YYYY-MM-DD HH:MM:SS' format. The supported range is '1000-01-01 00:00:00' 
to '9999-12-31 23:59:59'.

...

The TIMESTAMP data type has a range of '1970-01-01 00:00:01' UTC to '2038-01-09 03:14:07' UTC. It has varying 
properties, depending on the MySQL version and the SQL mode the server is running in.



In [ ]:
What if you want to keep the latest date and delete the rest of the records per id?



CREATE TABLE dummy1
	(`id` int, `name` varchar(5), `last_used_date` datetime)
;
	
INSERT INTO dummy1
	(`id`, `name`, `last_used_date`)
VALUES
	(1, 'name1', '2013-06-01 00:12:13'),
	(1, 'name1', '2013-06-01 18:00:13'),
	(2, 'name2', '2013-06-01 00:12:13'),
	(2, 'name2', '2013-07-01 00:12:13'),
	(2, 'name2', '2013-08-01 22:12:13'),
	(1, 'name1', '2013-09-01 00:12:13')
;



DELETE d
  FROM dummy1 d JOIN 
(
  SELECT d1.id, d1.name, MAX(d1.last_used_date) max_date
    FROM dummy1 d1 JOIN dummy1 d2
      ON d1.id = d2.id AND d1.name = d2.name
   GROUP BY id, name
) q ON d.id = q.id AND d.name = q.name AND d.last_used_date <> q.max_date;



In [ ]:

Timestamp/DateTime should never be used for any primary key the first issue being you can insert more than 1 record in 
second, leading to duplicate primary keys.

Use and integer with auto increment for your primary key then create a timestamp/Datetime and create an index on the 
column.

